In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV, LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import os
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, CuDNNLSTM, Input, concatenate, Concatenate
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
url = 'http://www.hec.unil.ch/agoyal/docs/PredictorData2017.xlsx'
mn_df = pd.read_excel(url,header=0,sheetname='Monthly', index_col='yyyymm')

### 도출해야 하는 variable list
1. **Equity Premium**: $ln(Index_{t+1} + D12_{t+1}) - ln(index_{t}) -ln(1+Rfree_{t+1})$
2. **dp** (Dividend Price Ratio): $ln(D12_{t})-ln(Index_{t})$
3. **dy** (Dividend Yield): $ln(D12_{t+1})-ln(Index_{t})$
4. **ep** (Earnings Price Ratio): $ln(E12_{t})-ln(Index_{t})$
5. **de** (Dividend Payout Ratio): $ln(D12_{t})-ln(E12_{t})$
6. **svar** (Stock Variance): $\sum{(Daily Return)^2}$ ***Given***
7. **bm** (Book to Market Ratio): $\frac{book value}{market value} for Dow Jones Industrial Average$ ***Given***
8. **ntis** (Net Equity Expansion): ratio of 12-month moving sums of net issues by NYSE listed stocks divided by the total end-of-year market capitalization of NYSE stocks ***Given*** <br> 
$Net Issue_{t} = Mcap_{t} - Mcap_{t-1}\times(1+vwretx_{t})$<br> 
where $Mcap$ is the total market capitalization and $vwretx$ is the value weighted return on the NYSE index
9. **tbl** (Treasury Bills): 3-Month Treasury BIll ***Given***
10. **ltr** (Long Term Rate of Returns): from Ibbotson's *Stocks, Bonds, Bills and Inflation Yearbook* ***Given***
11. **tms** (The Term Spread): Difference between the long term yield on government bonds and the Treasury-Bill $lty_{t}-tbl_{t}$
12. **dfy** (Default Yield Spread): Difference between BAA and AAA-rated orporate bond yields $BAA_{t}-AAA_{t}$
13. **dfr** (Default Return Spread): Difference between long-term corporate bond and long-term bovernment bond returns <br>
$corpr_{t}-ltr_{t}$
14. **infl** (Inflation): Consumer Price Index(***CPI***) from Bureau of Labor Statistics ***Given***

**Used Variables** : <br>
premium, dp, dy, ep, de, svar, bm, ntis, tbl, ltr, tms, dfy, dfr, infl

**Factor Model** <br>
$R_{t+1}=\alpha+\beta X_{t}+\beta_{f}F_{t}+\epsilon_{t+1}$ <br>
$F_{t}=F^{W,b}(X_{t})$

**Regression Model following Welch and Goyal(2008) ** <br>
$Equity\,Premium(t)=\gamma_{0}+\gamma_{1}\times x(t-1)+\epsilon(t)$

In [3]:
def variable_deriv(df):    
    #Yielding Equity Premium
    df['premium'] = np.log((df.Index+df.D12)/(df.Index.shift(periods=1)*(1+df.Rfree)))
    #Yielding Indicator where its previous Return was positive
    df['indicator'] = (df.premium.shift(periods=1)>0).apply(lambda x: 1 if x==True else 0)       
    #Yielding Dividend Price Ratio
    df['dp'] = np.log(df.D12) - np.log(df.Index)
    #Yielding Divident Yield
    df['dy'] = np.log(df.D12) - np.log(df.Index.shift(periods=1))
    #Yielding Earnings Price Ratio
    df['ep'] = np.log(df.E12) - np.log(df.Index)
    #Yielding Divident Payout Ratio
    df['de'] = np.log(df.D12) - np.log(df.E12)
    #Yielding The Term Spread
    df['tms'] = df.lty - df.tbl
    #Yielding The Default Yield Spread
    df['dfy'] = df.BAA - df.AAA
    #Yielding The Default Return Spread
    df['dfr'] = df.corpr - df.ltr
################################################################################################################
variable_deriv(mn_df)
#variable_deriv(qr_df)
#variable_deriv(yr_df)
################################################################################################################
variables = ['dp','dy','ep','de','svar','bm','ntis','tbl','ltr','tms','dfy','dfr','infl']
all_variables= ['premium','dp','dy','ep','de','svar','bm','ntis','tbl','ltr','tms','dfy','dfr','infl']
variables_sq=variables.copy()
variables_sq.extend([item+str('_sq') for item in variables])
variables_asy=variables.copy()
variables_asy.extend([item+str('_ind') for item in variables])
################################################################################################################
##Getting final variables from 1927.01~2017.12
def variable_transform(df, type_=None):
    ori=df.loc[192701:,all_variables]
    ind=df.loc[192701:,'indicator']
    #X 들
    if type_=="original" :
        ori.loc[:,variables] = StandardScaler().fit_transform(ori.loc[:,variables])
        return ori
    #X^2 들
    elif type_=="squared":
        output = pd.concat([ori,ori.apply(lambda x: x**2).add_suffix('_sq')],axis=1)
        output.loc[:,variables_sq] = StandardScaler().fit_transform(output.loc[:,variables_sq])
        return output
    #Asymmetric term
    elif type_=="asymmetric":    
        output = pd.concat([ori,ori.apply(lambda x: x*ind).add_suffix('_ind')],axis=1)
        output.loc[:,variables_asy] = StandardScaler().fit_transform(output.loc[:,variables_asy])
        return output
################################################################################################################
mn_df_ori=variable_transform(mn_df, type_='original')
mn_df_sqr=variable_transform(mn_df, type_='squared')
mn_df_asy=variable_transform(mn_df, type_='asymmetric')
################################################################################################################
class timeseries_cv():
    
    def __init__(self, n_split=3) :
        self.n_split = n_split
        
    def split(self, X, y, group=None) :
        n_split = self.n_split
        n_fold = n_split + 1
        max_len = X.shape[0]
        indices = np.arange(max_len)
        for index in range(1,n_fold) :
            yield (indices[:-index],indices[[-index]])
    
    def get_n_splits(self, X, y, group=None):
        return self.n_split    
################################################################################################################
def mse(y_true, y_pred):
    return np.average(np.square(y_true[[0]],y_pred[[0]]))
################################################################################################################
def r2_os(y_true,y_pred):
    r_value = pd.DataFrame(index=y_pred.index, columns=['numerator','denominator'])
    r_value['numerator'] = (y_true - y_pred)**2
    r_value['denominator'] = (y_true - y_true.expanding(min_periods=0).mean())**2
    r_square = 1 - (r_value['numerator'].sum())/(r_value['denominator'].sum())
    return r_square

### Getting $R^2$  and MSPE

In [5]:
# Function for Calculating Predicted Premium
def each_prediction(model,name,df,premium_array,r2_in_sample, type_, variables, param_grid, window_type, window_range, time_cv):        
    for item in range(1091-window_range):#(1091-window_range): 
        if window_type=='expanding':
            base = 0            
        else :
            base = item
        #############################################################################################################        
        ### Selecting Variables            
        X_train = (df.iloc[(base):(item+window_range),df.columns.get_indexer(variables)])
        X_test = (df.iloc[(item+window_range):(item+(window_range+1)),df.columns.get_indexer(variables)])
        Y_train = df['premium'].iloc[base+1:(item+(window_range+1))]
        #############################################################################################################        
        ### Fitting the model and generating prediction, r2         
        model.fit(X_train, Y_train)
        model_pred = model.predict(X_test)
        insample_pred = model.predict(X_train)
        r2 = r2_score(y_pred=insample_pred,y_true=Y_train)        
        #############################################################################################################        
        ### Saving the prediction result  & In Sample R2        
        if name != 'PLS':
            premium_array.iat[item,premium_array.columns.get_loc(name)] = model_pred[0]
            r2_in_sample.iat[item,r2_in_sample.columns.get_loc(name)] = r2
        elif name == 'PLS':
            premium_array.iat[item,premium_array.columns.get_loc(name)] = model_pred[0][0]
            r2_in_sample.iat[item,r2_in_sample.columns.get_loc(name)] = r2

In [6]:
# Function for Implementing Prediction for each method
def premium_calc(df,type_, window_type, window_range):
    #############################################################################################################        
    ### Variables assignment for calling ###
    variables = ['dp','dy','ep','de','svar','bm','ntis','tbl','ltr','tms','dfy','dfr','infl']
    time_cv = timeseries_cv(n_split=3)
    window_range = 12 * int(window_range)    
    if type_=='original':
        pass
    elif type_=='squared':
        variables.extend([item+str('_sq') for item in variables])
    elif type_=='asymmetrical':
        variables.extend([item+str('_ind') for item in variables])
    #############################################################################################################    
    ### Creating Empty DataFrame for storing Predicted Premium ###
    pred_premium = pd.DataFrame(index=df.iloc[window_range+1:,:].index, 
                                columns=['OLS','Ridge','PLS','Lasso','ElasticNet'])
    r2_in_sample = pd.DataFrame(index=df.iloc[window_range+1:,:].index, 
                                columns=['OLS','Ridge','PLS','Lasso','ElasticNet'])
    #############################################################################################################    
    ### Defining Prediction for each possible Methodology ###    
    # OLS
    print('Predicting OLS')
    each_prediction(LinearRegression(),'OLS',df, pred_premium,r2_in_sample, type_, variables, None, window_type, window_range, time_cv)
    # Ridge
    print('Predicting Ridge')
    each_prediction(RidgeCV(cv=time_cv),'Ridge',df, pred_premium, r2_in_sample, type_, variables, None, window_type, window_range, time_cv)
    # PLS
    print('Predicting PLS')
    each_prediction(PLSRegression(),'PLS',df, pred_premium, r2_in_sample, type_, variables, None, window_type, window_range, time_cv)
    # Lasso
    print('Predicting Lasso')
    each_prediction(LassoCV(cv=time_cv, max_iter=10000, tol=0.001),'Lasso',df, pred_premium, r2_in_sample, type_, variables, None, window_type, window_range, time_cv)
    # ElasticNet
    print('Predicting ElasticNet')
    each_prediction(ElasticNetCV(cv=time_cv,max_iter=10000, tol=0.001),'ElasticNet',df, pred_premium, r2_in_sample, type_, variables, None, window_type, window_range, time_cv)

    return pred_premium, r2_in_sample

In [ ]:
base_expanding_original_35yr_pred,base_expanding_original_35yr_r2 = premium_calc(mn_df_ori,type_='original', window_type='expanding', window_range=35)
base_expanding_original_35yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_expanding_original_pred_35yr.csv')
base_expanding_original_35yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_expanding_original_r2_35yr.csv')

In [ ]:
base_expanding_asymmetrical_35yr_pred,base_asymmetrical_original_35yr_r2 = premium_calc(mn_df_asy,type_='asymmetrical', window_type='expanding', window_range=35)
base_expanding_asymmetrical_35yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_expanding_asymmetrical_pred_35yr.csv')
base_expanding_asymmetrical_35yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_expanding_asymmetrical_r2_35yr.csv')

In [ ]:
base_expanding_square_35yr_pred,base_expanding_square_35yr_r2 = premium_calc(mn_df_sqr,type_='squared', window_type='expanding', window_range=35)
base_expanding_square_35yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_expanding_square_pred_35yr.csv')
base_expanding_square_35yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_expanding_square_r2_35yr.csv')

In [ ]:
base_moving_original_20yr_pred,base_moving_original_20yr_r2 = premium_calc(mn_df_ori,type_='original', window_type='moving', window_range=20)
base_moving_original_20yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_original_pred_20yr.csv')
base_moving_original_20yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_original_r2_20yr.csv')

In [ ]:
base_moving_square_20yr_pred,base_moving_square_20yr_r2 = premium_calc(mn_df_sqr,type_='squared', window_type='moving', window_range=20)
base_moving_square_20yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_square_pred_20yr.csv')
base_moving_square_20yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_square_r2_20yr.csv')

In [ ]:
base_moving_asymmetrical_20yr_pred,base_moving_asymmetrical_20yr_r2 = premium_calc(mn_df_asy,type_='asymmetrical', window_type='moving', window_range=20)
base_moving_asymmetrical_20yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_asymmetrical_pred_20yr.csv')
base_moving_asymmetrical_20yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_asymmetrical_r2_20yr.csv')

In [ ]:
base_moving_original_10yr_pred,base_moving_original_10yr_r2 = premium_calc(mn_df_ori,type_='original', window_type='moving', window_range=10)
base_moving_original_10yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_original_pred_10yr.csv')
base_moving_original_10yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_original_r2_10yr.csv')

In [ ]:
base_moving_square_10yr_pred,base_moving_square_10yr_r2 = premium_calc(mn_df_sqr,type_='squared', window_type='moving', window_range=10)
base_moving_square_10yr_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_square_pred_10yr.csv')
base_moving_square_10yr_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_square_r2_10yr.csv')

In [ ]:
base_moving_asymmetrical_10yr_pred,base_moving_asymmetrical_10yr_r2 = premium_calc(mn_df_asy,type_='asymmetrical', window_type='moving', window_range=10)
base_moving_asymmetrical_pred.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_asymmetrical_pred_10yr.csv')
base_moving_asymmetrical_r2.to_csv(r'C:\Users\Home PC\Desktop\재무대학원\재무 대학원\논문\result\new\base\base_moving_asymmetrical_r2_10yr.csv')